In [39]:
import os
# os.getcwd()
os.chdir('/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/src')
import sys
sys.path.append("../") # go to parent dir

from src.wind_stf.pipelines.data_science.nodes import scale_timeseries, _get_last_train_idx, _split_train_test
from src.wind_stf.pipelines.data_engineering.nodes import build_power_centroids_mts

import pandas as pd
import pickle
from sklearn.preprocessing import QuantileTransformer
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
def unpickle(fp: str):
    with open(fp, 'rb') as handle:
        b = pickle.load(handle)
        
    return b

In [4]:
df_spatiotemporal = pd.read_hdf('../data/05_model_input/df_spatiotemporal.hdf')
cv_splits_dict = unpickle('../data/05_model_input/cv_splits_dict.pkl/2020-09-25T01.03.57.441Z/cv_splits_dict.pkl')
target_timeseries = ['DEF0C', 'DE111']

In [12]:
tss = df_spatiotemporal['temporal']

# get training dataset: based on the longest CV pass
last_train_idx = _get_last_train_idx(tss, cv_splits_dict)
tss_train = tss.loc[:last_train_idx, :]

# generate the transformer: Quantile Transformation + Level Offset (bias)
quantile_transformer = QuantileTransformer(
    output_distribution='normal',
    random_state=0
)

quantile_transformer.fit( tss_train )

level_offset = abs(
    quantile_transformer.transform(
        tss_train
    ).min()
)

# apply the transformer
tss_scaled = pd.DataFrame(
    index=tss.index,
    columns=tss.columns,
    data=quantile_transformer.transform(tss) + level_offset
)

In [13]:
df_spatiotemporal['temporal']

district,DE111,DE114,DE115,DE116,DE118,DE119,DE11A,DE11B,DE11C,DE11D,...,DEG0E,DEG0F,DEG0G,DEG0I,DEG0J,DEG0K,DEG0L,DEG0M,DEG0N,DEG0P
var,power,power,power,power,power,power,power,power,power,power,...,power,power,power,power,power,power,power,power,power,power
2013-01-01,0.178783,0.269458,0.351130,0.184231,0.357407,0.410291,0.362935,0.344386,0.284140,0.360815,...,0.187667,0.363388,0.393872,0.482197,0.551744,0.435241,0.372426,0.464629,0.393286,0.497463
2013-01-02,0.030363,0.063571,0.103089,0.030433,0.078740,0.108224,0.105199,0.093845,0.083709,0.093448,...,0.073910,0.133930,0.226143,0.205528,0.284261,0.219417,0.215123,0.272299,0.203758,0.248020
2013-01-03,0.041567,0.229298,0.182997,0.090303,0.460150,0.516133,0.410587,0.371699,0.320095,0.357628,...,0.107586,0.297686,0.451200,0.379433,0.536884,0.463397,0.424858,0.528489,0.426369,0.502846
2013-01-04,0.128148,0.205830,0.347322,0.144561,0.264061,0.319020,0.357322,0.220658,0.295479,0.364025,...,0.124282,0.342673,0.427804,0.447503,0.505166,0.311061,0.373373,0.438596,0.329357,0.347780
2013-01-05,0.126854,0.346798,0.274581,0.188337,0.212976,0.221080,0.343575,0.272172,0.459154,0.448515,...,0.294815,0.357460,0.321136,0.415867,0.361462,0.349354,0.301562,0.275772,0.331298,0.337855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,0.026242,0.029231,0.024730,0.027717,0.099735,0.090871,0.093439,0.178864,0.035651,0.064401,...,0.053993,0.165269,0.185665,0.306531,0.413768,0.415238,0.415388,0.478009,0.344419,0.390953
2015-12-28,0.006859,0.085662,0.021305,0.011488,0.030665,0.018439,0.093263,0.040121,0.124893,0.127186,...,0.004232,0.019065,0.030037,0.043915,0.063921,0.066541,0.067548,0.095427,0.062628,0.054857
2015-12-29,0.001484,0.019175,0.006363,0.003324,0.011347,0.007789,0.042832,0.033454,0.031868,0.039469,...,0.024023,0.034685,0.016879,0.070263,0.035795,0.166228,0.043741,0.032034,0.077701,0.072997


In [14]:
tss_scaled

district,DE111,DE114,DE115,DE116,DE118,DE119,DE11A,DE11B,DE11C,DE11D,...,DEG0E,DEG0F,DEG0G,DEG0I,DEG0J,DEG0K,DEG0L,DEG0M,DEG0N,DEG0P
var,power,power,power,power,power,power,power,power,power,power,...,power,power,power,power,power,power,power,power,power,power
2013-01-01,6.731783,6.331834,6.697029,6.617252,6.595000,6.640230,6.552457,6.640230,6.266212,6.505730,...,6.391434,6.827878,6.602340,6.870906,6.797481,6.655982,6.505730,6.680310,6.602340,6.697029
2013-01-02,5.286310,5.206232,5.633092,5.186928,5.561445,5.491612,5.426005,5.529239,5.197959,5.314028,...,5.675915,5.920439,6.155809,6.040173,6.121516,5.990168,6.048053,6.125743,5.990168,6.040173
2013-01-03,5.526329,6.228017,6.096491,6.020698,6.859859,6.917272,6.688618,6.705548,6.369280,6.499284,...,5.956680,6.559377,6.787670,6.538807,6.768493,6.768493,6.731783,6.849010,6.705548,6.731783
2013-01-04,6.414190,6.108932,6.680310,6.449552,6.391434,6.363830,6.538807,6.223348,6.300957,6.512231,...,6.059973,6.740767,6.705548,6.778009,6.680310,6.276007,6.518788,6.532074,6.437592,6.353033
2013-01-05,6.397065,6.595000,6.443550,6.648061,6.200327,6.055986,6.512231,6.391434,6.740767,6.731783,...,6.849010,6.787670,6.425807,6.624828,6.337086,6.385841,6.326613,6.138525,6.443550,6.300957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,5.199008,4.595032,4.477878,5.121189,5.698143,5.335572,5.296462,6.043009,4.545007,5.034706,...,5.461269,6.095548,6.032712,6.344166,6.442358,6.616471,6.692499,6.718401,6.470230,6.454886
2015-12-28,4.267920,5.426542,4.364393,4.465407,4.923340,4.183142,5.295198,4.880368,5.550668,5.596479,...,3.709637,4.558709,4.690882,4.853250,5.023833,4.981143,5.110982,5.218306,5.078171,4.824936
2015-12-29,3.181828,4.330976,3.374142,3.627163,4.318599,3.586713,4.666383,4.779955,4.458903,4.601966,...,4.819777,4.969116,4.314396,5.179636,4.578945,5.746063,4.823293,4.412612,5.214873,5.125337


In [16]:
df_spatiotemporal['temporal'].update( tss_scaled )

district,DE111,DE114,DE115,DE116,DE118,DE119,DE11A,DE11B,DE11C,DE11D,...,DEG0E,DEG0F,DEG0G,DEG0I,DEG0J,DEG0K,DEG0L,DEG0M,DEG0N,DEG0P
var,power,power,power,power,power,power,power,power,power,power,...,power,power,power,power,power,power,power,power,power,power
2013-01-01,6.731783,6.331834,6.697029,6.617252,6.595000,6.640230,6.552457,6.640230,6.266212,6.505730,...,6.391434,6.827878,6.602340,6.870906,6.797481,6.655982,6.505730,6.680310,6.602340,6.697029
2013-01-02,5.286310,5.206232,5.633092,5.186928,5.561445,5.491612,5.426005,5.529239,5.197959,5.314028,...,5.675915,5.920439,6.155809,6.040173,6.121516,5.990168,6.048053,6.125743,5.990168,6.040173
2013-01-03,5.526329,6.228017,6.096491,6.020698,6.859859,6.917272,6.688618,6.705548,6.369280,6.499284,...,5.956680,6.559377,6.787670,6.538807,6.768493,6.768493,6.731783,6.849010,6.705548,6.731783
2013-01-04,6.414190,6.108932,6.680310,6.449552,6.391434,6.363830,6.538807,6.223348,6.300957,6.512231,...,6.059973,6.740767,6.705548,6.778009,6.680310,6.276007,6.518788,6.532074,6.437592,6.353033
2013-01-05,6.397065,6.595000,6.443550,6.648061,6.200327,6.055986,6.512231,6.391434,6.740767,6.731783,...,6.849010,6.787670,6.425807,6.624828,6.337086,6.385841,6.326613,6.138525,6.443550,6.300957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,5.199008,4.595032,4.477878,5.121189,5.698143,5.335572,5.296462,6.043009,4.545007,5.034706,...,5.461269,6.095548,6.032712,6.344166,6.442358,6.616471,6.692499,6.718401,6.470230,6.454886
2015-12-28,4.267920,5.426542,4.364393,4.465407,4.923340,4.183142,5.295198,4.880368,5.550668,5.596479,...,3.709637,4.558709,4.690882,4.853250,5.023833,4.981143,5.110982,5.218306,5.078171,4.824936
2015-12-29,3.181828,4.330976,3.374142,3.627163,4.318599,3.586713,4.666383,4.779955,4.458903,4.601966,...,4.819777,4.969116,4.314396,5.179636,4.578945,5.746063,4.823293,4.412612,5.214873,5.125337


In [10]:
df_temporal_out = pd.DataFrame(
    columns= pd.MultiIndex.from_product(
        [tss_scaled.columns, ['power']],
        names=['nuts_id', 'var']
    ),
    index=tss_scaled.index,
    data=tss_scaled.values,
)

In [11]:
df_temporal_out

nuts_id,DEF0C,DE111
var,power,power
2013-01-01,5.602990,6.731783
2013-01-02,5.526329,5.286310
2013-01-03,6.480263,5.526329
2013-01-04,6.617252,6.414190
2013-01-05,4.892643,6.397065
...,...,...
2015-12-27,6.809781,5.199008
2015-12-28,5.657192,4.267920
2015-12-29,6.602635,3.181828


In [68]:
df_st_pp = pd.read_hdf('../data/05_model_input/df_spatiotemporal_preprocessed.hdf')

In [69]:
(df_st_pp['temporal']==0).sum().sum()

0

In [24]:
cv_splits_dict

{'pass_1': {'train_idx': [0, 730], 'test_idx': [730, 737]},
 'pass_2': {'train_idx': [0, 820], 'test_idx': [820, 827]},
 'pass_3': {'train_idx': [0, 910], 'test_idx': [910, 917]}}

In [99]:
modeling = {
    'mode': 'temporal',
    'approach': 'HW-ES',
    'hyperpars': {
        'trend': 'additive',
        'seasonal': 'multiplicative',
        'seasonal_periods': 365,
    },
    'targets': ['DEF0C', 'DE111'],
}

In [100]:
hyperpars = modeling['hyperpars']
targets = modeling['targets']
mode = modeling['mode']

In [70]:
df = df_st_pp

In [71]:
if mode == 'temporal':
    df = df['temporal']

In [72]:
df = df[targets]

In [73]:
df

district,DEF0C,DE111
var,power,power
2013-01-01,5.602990,6.731783
2013-01-02,5.526329,5.286310
2013-01-03,6.480263,5.526329
2013-01-04,6.617252,6.414190
2013-01-05,4.892643,6.397065
...,...,...
2015-12-27,6.809781,5.199008
2015-12-28,5.657192,4.267920
2015-12-29,6.602635,3.181828


In [76]:
model = {}
for pass_id in cv_splits_dict.keys():

    model[pass_id] = {}

    # splitting
    y = _split_train_test(df, cv_splits_dict, pass_id)

    # training
    for district in df.columns:
        model[pass_id][district] = ExponentialSmoothing(
            y['train'][district],
            **hyperpars,
        ).fit()

/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/venv/lib/python3.8/site-packages/statsmodels/tsa/holtwinters.py:743: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warn("Optimization failed to converge. Check mle_retvals.",
/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/venv/lib/python3.8/site-packages/statsmodels/tsa/holtwinters.py:743: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warn("Optimization failed to converge. Check mle_retvals.",
/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/venv/lib/python3.8/site-packages/statsmodels/tsa/holtwinters.py:743: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warn("Optimization failed to converge. Check mle_retvals.",
/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/venv/lib/python3.8/site-packages/statsmodels/tsa/holtwinters.py:743: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warn("Optimi

In [80]:
y['train'][district]

2013-01-01    6.731783
2013-01-02    5.286310
2013-01-03    5.526329
2013-01-04    6.414190
2013-01-05    6.397065
                ...   
2015-06-25    4.018391
2015-06-26    3.978697
2015-06-27    5.588076
2015-06-28    4.137322
2015-06-29    4.338702
Freq: D, Name: (DE111, power), Length: 910, dtype: float64

In [85]:
model['pass_2'][('DEF0C', 'power')].predict()

2015-04-01    7.806444
Freq: D, dtype: float64

In [86]:
cv_splits_dict

{'pass_1': {'train_idx': [0, 730], 'test_idx': [730, 737]},
 'pass_2': {'train_idx': [0, 820], 'test_idx': [820, 827]},
 'pass_3': {'train_idx': [0, 910], 'test_idx': [910, 917]}}

In [95]:
y['train'].iloc[820]

district  var  
DEF0C     power    6.624828
DE111     power    7.819031
Name: 2015-04-01 00:00:00, dtype: float64

In [96]:
n_passes = 3

In [97]:
for pass_ in range(n_passes):
    print(pass_)

0
1
2


In [108]:
test_idx = y['test'].index

pred_scaled = {}
pred = {}

for pass_id in cv_splits_dict.keys():  
    
    for district in targets:
        # prediction
        pred_scaled[pass_id] = {
            district: model[pass_id][(district, 'power')].predict(
                start=test_idx[0],
                end=test_idx[-1]
            )
        }
    
    # postprocessing prediction
    pred[pass_id] = pd.DataFrame(
        data =  quantile_transformer.inverse_transform(
            pd.DataFrame( pred_scaled[pass_id] ) - 10
        ),
        columns = tss_scaled.columns,
        index = test_idx,
    )

ValueError: X does not have the same number of features as the previously fitted data. Got 1 instead of 292.